In [ ]:
from Grid_to_Graph.GRDECL_Parser import GRDECL_Parser
from Grid_to_Graph.struct_discretizer import StructDiscretizer
import numpy as np
import graph_coarsening.graph_utils as graph_utils
#import graph_coarsening.useful_custom_functions as useful_custom_functions
from graph_coarsening.coarsening_utils import *
from collections import defaultdict
from operator import itemgetter
import pandas as pd
import fileinput
import sys
import os

%matplotlib inline
# %matplotlib notebook
# %pylab

In [ ]:
def OneDim_to_ThreeDim_Converter(i_max, j_max, k_max, One_Dim):
    # приводим массив к 3D размерности грида.
    Three_Dim = np.array(np.zeros((i_max, j_max, k_max)))
    i = 0
    j = 0
    k = 0
    z = 0

    while k <= k_max - 1:
        while j <= j_max - 1:
            while i <= i_max - 1:
                Three_Dim[i, j, k] = One_Dim[z]
                i += 1
                z += 1
            i = 0
            j += 1
        j = 0
        k += 1

    return Three_Dim


def get_coarsening(Gall, Call, size=5, node_size=20, title=""):
    edges_s = {}

    n_levels = len(Gall) - 1
    for level in range(n_levels):
        G = Gall[level]
        edges = np.array(G.get_edge_list()[0:2])

        Gc = Gall[level + 1]
        edges_c = np.array(Gc.get_edge_list()[0:2])
        C = Call[level]
        C = C.toarray()

        for i in range(Gc.N):
            subgraph = np.arange(G.N)[C[i, :] > 0]
            edges_s[i] = subgraph
            #if np.clip(len(subgraph) - 1, 0, 4) > 0:
            #edges_s[i] = subgraph

    return edges_s


def replaceAll(file, searchExp, replaceExp):
    for line in fileinput.input(file, inplace=1):
        if searchExp in line:
            line = line.replace(searchExp,replaceExp)
        sys.stdout.write(line)

Считываем данные по гриду

In [ ]:
grid_file = 'data/3Dgrid_5000.grdecl'

replaceAll(grid_file,'-- Generated : Petrel'," ")

grid_data = GRDECL_Parser(grid_file)

In [ ]:
grid_data.NX,grid_data.NY, grid_data.NZ 

In [ ]:
dx = 8 
dy = 8
dz = 4



actnum = grid_data.ACTNUM
nx = grid_data.NX
ny = grid_data.NY
nz = grid_data.NZ

coord = grid_data.COORD

zcorn = grid_data.ZCORN

permx = grid_data.SpatialDatas['PERMX'] * 100
permx[permx<=0] = 0.0002
permy = permx
permz = permy

poro = grid_data.SpatialDatas['PORO']
poro[poro<=0] = 0.0002

volume = np.zeros_like(poro)
volume.fill(dx*dy*dz)

depth = np.array([zcorn[i-8:i].mean() for i in np.arange(8, len(zcorn)+1, 8)])

n = nx * ny * nz

In [ ]:

# Визуализируем первый грид на первом subplot
def vis(grid1):
    fig, (ax1, ax2) = plt.subplots(1, 2, subplot_kw={'projection': '3d'}, figsize=(6, 6))

    # Визуализируем первый грид на первом subplot
    x, y, z = np.meshgrid(np.arange(grid1.shape[0]),
                          np.arange(grid1.shape[1]),
                          np.arange(grid1.shape[2]))
    print(x.ravel())
    
    ax1.scatter(x.ravel(), y.ravel(), z.ravel(), c=grid1.flatten(), vmin=0, vmax=1)

    plt.show()


vis(discretizer.convert_to_3d_array(poro, 'PORO'))

In [ ]:
discretizer.convert_to_3d_array(poro, 'PORO').ravel()

Получаем conection list

In [ ]:
discretizer = StructDiscretizer(nx=nx, ny=ny, nz=nz, dx=dx, dy=dy, dz=dz, permx=permx, permy=permy,
                                             permz=permz, coord=coord, zcorn=zcorn)

if discretizer.is_cpg:
    cell_m, cell_p, tran, tran_thermal = discretizer.calc_cpg_discr()
else:
    cell_m, cell_p, tran, tran_thermal = discretizer.calc_structured_discr()

In [ ]:
# apply actnum filter if needed - all arrays providing a value for a single grid block should be passed
op_num = 0
arrs = [poro, depth, volume, op_num]

cell_m, cell_p, tran, tran_thermal, arrs_local = discretizer.apply_actnum_filter(actnum, cell_m,
                                                                                 cell_p, tran,
                                                                                 tran_thermal, arrs)
poro, depth, volume, op_num = arrs_local

Создаём граф

In [ ]:
# n = np.unique(cell_m).shape[0] + 1
# graph_W = np.zeros((n, n))
# for i, j, t in zip(cell_m, cell_p, tran):
#     if t > 0:
#         w = 1/t
#     else:
#         w = 1/0.0001
#     graph_W[i][j] = w
#     graph_W[j][i] = w

edges = list(zip(cell_m, cell_p))
weights = [1/t if t>0 else 1/0.0001 for t in tran]

# Количество вершин в графе
num_vertices = len(set(sum(edges, ())))

# Создание матрицы смежности
adjacency_matrix = np.zeros((num_vertices, num_vertices))
for edge, weight in zip(edges, weights):
    adjacency_matrix[edge[0], edge[1]] = weight
    adjacency_matrix[edge[1], edge[0]] = weight

In [ ]:
coords = np.array(OneDim_to_ThreeDim_Converter(nx, ny, nz, np.arange(n) - 1))
coords_new = np.zeros((n, 3))
for i in range(nx):
    for j in range(ny):
        for k in range(nz):
            if coords[i, j, k] != -1:
                coords_new[int(coords[i, j, k])] = np.array([i, j, k])

In [ ]:
N = coords_new.shape[0]

G = graphs.Graph(adjacency_matrix)
G.set_coordinates(coords_new)

N = G.N
L = G.L.toarray()
S = graph_utils.get_S(G).T

In [ ]:
G

Загрубляем граф

In [ ]:
method = 'heavy_edge'

# Parameters
r = 0.45   # the extend of dimensionality reduction (r=0 means no reduction)
k = 3
kmax = int(3 * k)

C, Gc, Call, Gall = coarsen(G, K=k, r=r, method=method)
#metrics = coarsening_quality(G, C, kmax=kmax)
n = Gc.N

# print('{:16} | r: {:1.4}, nedges: {}, levels: {}, epsilon: {:1.4}'.format(method, metrics['r'], metrics['m'], len(Call),
#                                                                           metrics['error_subspace'][k - 1]))

In [ ]:
method = 'heavy_edge'

# Parameters
r = 0.5   # the extend of dimensionality reduction (r=0 means no reduction)
k = 5
kmax = int(3 * k)

C, Gc, Call, Gall, coarsening_list, iC, err_list, r_list = coarsen(G, K=k, r=r, method=method, 
                            #algorithm = "optimal"
                            )
#metrics = coarsening_quality(G, C, kmax=kmax)
n = Gc.N

# print('{:16} | r: {:1.4}, nedges: {}, levels: {}, epsilon: {:1.4}'.format(method, metrics['r'], metrics['m'], len(Call),
#                                                                           metrics['error_subspace'][k - 1]))

metrics = coarsening_quality(G, C, kmax=k)
np.nanmean(metrics['error_eigenvalue']) 

In [ ]:
plt.plot(r_list, err_list, '-')


In [ ]:
poro_d = poro
perm_d = permx

poro_d_2 = coarsen_vector(poro_d, C)
perm_d_2 = coarsen_vector(perm_d, C)
depth_d_2 = coarsen_vector(depth, C)

In [ ]:
Gall[0].plot_signal(poro_d)
Gall[1].plot_signal(poro_d_2)

In [ ]:
Gall[0].plot_signal(perm_d)
Gall[1].plot_signal(perm_d_2)

In [ ]:
g_dict = get_coarsening(Gall, Call, size=5)
num_of_cells = np.array([len(g) for g in g_dict.values()])

In [ ]:
g_dict